<a href="https://colab.research.google.com/github/Vaishnavi263050/Assignment1/blob/main/hackathonnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()



Saving hacktest.csv to hacktest (1).csv


In [16]:
# Step 1: Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report



In [7]:
train_df = pd.read_csv('hacktrain.csv')
test_df = pd.read_csv('hacktest.csv')

# Preview to confirm
train_df.head(), test_df.head()
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (8000, 30)
Test shape: (2845, 29)


In [8]:
# Step 3: Basic Preprocessing
# Handle missing values by filling with zeros (or use better imputation if needed)
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

ndvi_cols = [col for col in train_df.columns if '_N' in col or 'NDVI' in col]





In [9]:
def add_ndvi_features(df):
    df['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    df['ndvi_std'] = df[ndvi_cols].std(axis=1)
    df['ndvi_max'] = df[ndvi_cols].max(axis=1)
    df['ndvi_min'] = df[ndvi_cols].min(axis=1)
    df['ndvi_range'] = df['ndvi_max'] - df['ndvi_min']
    return df

train_df = add_ndvi_features(train_df)
test_df = add_ndvi_features(test_df)


In [17]:
X_train = train_df.drop(['ID', 'class'], axis=1)
X_test = test_df.drop(['ID'], axis=1)


le = LabelEncoder()
y_train = le.fit_transform(train_df['class'])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)




In [18]:
 model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
model.fit(X_tr, y_tr)


y_val_pred = model.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", val_acc)
print("Classification Report:\n", classification_report(y_val, y_val_pred, target_names=le.classes_))


Validation accuracy: 0.966875
Classification Report:
               precision    recall  f1-score   support

        farm       0.81      0.95      0.87       161
      forest       1.00      1.00      1.00      1231
       grass       0.92      0.81      0.86        43
  impervious       0.91      0.79      0.84       141
     orchard       1.00      0.17      0.29         6
       water       1.00      0.89      0.94        18

    accuracy                           0.97      1600
   macro avg       0.94      0.77      0.80      1600
weighted avg       0.97      0.97      0.97      1600



In [19]:
y_test_pred = model.predict(X_test)



In [20]:
 submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'class': le.inverse_transform(y_test_pred)
})
submission_df.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv!")
from google.colab import files
files.download('submission.csv')


Submission file saved as submission.csv!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>